<a href="https://www.kaggle.com/code/mesutssmn/book-price-prediction?scriptVersionId=167043724" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
df = pd.read_csv("/kaggle/input/books-sold-dataset/Kaggle_Books-Selling-Records-.csv")

In [ ]:
df.head()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.drop(['Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50',
       'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54',
       'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58',
       'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62',
       'Unnamed: 63', 'Unnamed: 64', 'Unnamed: 65', 'Unnamed: 66',
       'Unnamed: 67'], axis = 1, inplace = True)

In [ ]:
df_new = df.drop(["Gender\nF=1665\nM=1815", "Item-Status", "Customer ID",'Purchase-date',
         "Product-Name",'Date', 'Time', 'Quarter','Month', 'Date.1',
         'Special Day','Day', 'Morning', 'Afternoon', 'Evening', 'Night', 'Product-Name','Currency','Ship-City',
         'Ship-State', 'Ship-Postal-Code', 'Publication', 'Author ','Online Sale Offers '], axis = 1)

In [ ]:
df.head(3)

In [ ]:
df_new["Category"].unique()

In [ ]:
df_new["Category"] = df_new["Category"].map({'college':0, 'competition ':1, 'School':2, 'school':2})
df_new["Category"].unique()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df['Item-Status'].unique()

In [ ]:
df['Quantity'].unique()

In [ ]:
df['Category'].unique()

In [ ]:
df.columns

In [ ]:
df['Online Sale Offers '].unique()

In [ ]:
df['Author '].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df_new.corr()

In [ ]:
plt.figure(figsize=(13,10))
sns.heatmap(df_new.corr(), annot=True, fmt='.2f', cmap="magma")

In [ ]:
df.head()

In [ ]:
df['Purchase-date'] = df['Purchase-date'].str[0:10]

In [ ]:
df["Purchase-date"] = pd.to_datetime(df["Purchase-date"], format = "%Y-%m-%d")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
sns.lineplot(data = df.head(160), x = "Purchase-date", y = "Cost Price", color = "green", hue = "Gender")
plt.show()

In [ ]:
sns.lineplot(data = df.head(160), x = "Purchase-date", y = "Item Price",hue = "Gender",
             palette = "magma")

In [ ]:
sns.scatterplot(df,  x = "Purchase-date", y = "Item Price",hue = "Gender")

In [ ]:
sns.barplot(df, x = "Category", y = "Profit (INR)")

In [ ]:
df["Category"].value_counts()

In [ ]:
sns.histplot(df["Profit (INR)"])

In [ ]:
sns.histplot(df["Cost Price"])

In [ ]:
df.groupby(by = ['Category']).count()

In [ ]:
sns.countplot(df, x = 'Category', hue = "Gender")

In [ ]:
sns.barplot(df, x='Category', y='Cost Price', hue='Gender')

In [ ]:
sns.barplot(df, x='Category', y='Gender')

In [ ]:
df.columns

In [ ]:
sns.catplot(df, y = "Category", x = "Cost Price", hue = "Gender", palette = "magma")

In [ ]:
sns.catplot(df, y = "Category", x = "Cost Price", hue = "Gender", palette = "magma", kind = "box")

In [ ]:
sns.pairplot(df_new, kind="reg", hue = "Gender")

In [ ]:
df_new.info()

In [ ]:
df_new["Category"] = df_new["Category"].map({0:'college', 1:'competition ', 2:'School'})
df_new["Category"].unique()

In [ ]:
df_new.head()

In [ ]:
df_dummies=pd.get_dummies(df_new, columns=['Category'], drop_first=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

In [ ]:
X = df_dummies.drop('Cost Price', axis=1)
y = df_dummies['Cost Price']

In [ ]:
X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=0.25, random_state=40)

In [ ]:
lr = LinearRegression()
model = lr.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
xgb = XGBRegressor()
model = xgb.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
xgb = XGBRegressor()
model_scaled = xgb.fit(X_train_scaled, y_train)
model_scaled.score(X_test_scaled, y_test)

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.Sequential()
input_layer = tf.keras.layers.Dense(100, input_shape=(X_train_scaled.shape[1],), activation="relu")
model.add(input_layer)

model.add(tf.keras.layers.Dense(50, activation="relu"))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1, activation="linear"))

model.summary()
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mape'])
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True)

In [ ]:
history = model.fit(X_train_scaled, y_train,
                    callbacks=[early_stopping],
                    epochs=500, verbose=1, batch_size=2000, validation_split=0.2)

In [ ]:
model_y_pred = model.predict(X_test_scaled)
r2 = r2_score(y_test, model_y_pred)
print('R2 score:', r2)

In [ ]:
plt.figure(figsize=(10, 6))

train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss, label="Training Loss")
plt.plot(epochs, val_loss, label="Validation Loss")

plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()

In [ ]:
ann = tf.keras.Sequential([tf.keras.layers.Dense(64, activation="relu"),
                           tf.keras.layers.Dense(16, activation="relu"),
                 tf.keras.layers.Dense(1, activation='linear')])

ann.compile(optimizer='adam',
            loss='mean_absolute_error',
            metrics=['mape'])


In [ ]:
history = ann.fit(X_train_scaled, y_train, epochs=200, validation_split=0.2)

In [ ]:
ann_y_pred = ann.predict(X_test_scaled)
r2 = r2_score(y_test, ann_y_pred)
print('R2 score:', r2)

In [ ]:
plt.figure(figsize=(10, 6))

train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss, label="Training Loss")
plt.plot(epochs, val_loss, label="Validation Loss")

plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()